## CBOE SKEW analysis

In [ ]:
import sys,os

if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
from volgrid import dgrid
from volgrid import create_voltables as cvt
import dash_core_components as dcc
import traceback

import plotly.graph_objs as go
from plotly.offline import iplot
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import numpy as np
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as pdr
import datetime
import pytz

#  do rest of imports
import dash
import dash_html_components as html
from dash.dependencies import Input, Output,State
import yfinance as yf
import pathlib
import pg_pandas as pg
from dateutil.relativedelta import *

### Define some interest rate and calendar functions

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar
bday_us = pd.offsets.CustomBusinessDay(calendar=USFederalHolidayCalendar())
TIMEZONE = 'US/Eastern'

def get_rate_df(num_months_for_rate=1,start_datetime=None,end_datetime=None):
    # see if you can get Libor from the FRED API
    if start_datetime is None:
        n = datetime.datetime.now() - 252*bday_us #datetime.timedelta(7)
    else:
        if type(start_datetime)==int:
            sdt = start_datetime
            n = datetime.datetime(int(str(sdt)[0:4]),int(str(sdt)[4:6]), int(str(sdt)[6:8]))
        else:
            n = start_datetime 
    y = n.year
    m = n.month
    d = n.day
    beg = '%04d-%02d-%02d' %(y,m,d)
    ed = end_datetime if end_datetime is not None else (datetime.datetime.now() - datetime.timedelta(1))
    if type(ed)==int:
        ed = datetime.datetime(int(str(ed)[0:4]), int(str(ed)[4:6]), int(str(ed)[6:8]))
    y = ed.year
    m = ed.month
    d = ed.day
    eds = '%04d-%02d-%02d' %(y,m,d)
    fred_libor_file = f'USD{num_months_for_rate}MTD156N'
    df = pdr.DataReader(fred_libor_file, "fred", f'{beg}', f'{eds}')
    if len(df)<1:
        raise ValueError(f'FRED calendar of {fred_libor_file} does not contain dates {beg} through {eds}')
    df['fixed_rate'] = df[f'USD{num_months_for_rate}MTD156N'].astype(float)/100
    df.columns = df.columns.get_level_values(0)
    df['date_yyyymmdd'] = [int(d.year*100*100) + int(d.month*100) + int(d.day) for d in df.index]
    df.index = range(len(df))
    df['prate'] = df.shift(1).fixed_rate
    df['fixed_rate'] = df.apply(lambda r:r.prate if r.fixed_rate !=r.fixed_rate else r.fixed_rate,axis=1)
    return df[['date_yyyymmdd','fixed_rate']]


def get_rate(num_months_for_rate=1,rate_datetime=None):
    # see if you can get Libor from the FRED API
    if rate_datetime is None:
        n = datetime.datetime.now() - 7*bday_us #datetime.timedelta(7)
    else:
        n = rate_datetime #- datetime.timedelta(14)
    y = n.year
    m = n.month
    d = n.day
    beg = '%04d-%02d-%02d' %(y,m,d)
#     ed = n  + datetime.timedelta(1)
    ed = datetime.datetime.now()
    y = ed.year
    m = ed.month
    d = ed.day
    eds = '%04d-%02d-%02d' %(y,m,d)
    fred_libor_file = f'USD{num_months_for_rate}MTD156N'
    df = pdr.DataReader(fred_libor_file, "fred", f'{beg}', f'{eds}')
    if len(df)<1:
        raise ValueError(f'FRED calendar of {fred_libor_file} does not contain dates {beg} through {eds}')
#     fixed_rate = float(df.iloc[len(df)-1][f'USD{num_months_for_rate}MTD156N'])/100
    fixed_rate = float(df.iloc[-1][f'USD{num_months_for_rate}MTD156N'])/100
    return fixed_rate

def get_nth_weekday(year,month,target_weekday,nth_occurrence):
    '''
    weekday is the term that assigns numbers from 0 to 6 to the days of the weeks.
    weekday 0 = monday
    '''
    # get dayofweeks of year,month,1
    weekday_01 = datetime.datetime(year,month,1).weekday()
    if weekday_01 <= target_weekday:
        day_of_month_of_first_occurence = target_weekday - weekday_01
        day_of_month_of_nth_occurence = day_of_month_of_first_occurence + 1 + (nth_occurrence - 1) * 7
    else:
        day_of_month_of_nth_occurence = target_weekday - weekday_01 + 1 + (nth_occurrence) * 7 
    return datetime.datetime(year,month,day_of_month_of_nth_occurence)


MONTH_CODES = 'FGHJKMNQUVXZ'
DICT_MONTH_CODE = {MONTH_CODES[i]:i+1 for i in range(len(MONTH_CODES))}


def get_ES_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return get_nth_weekday(year,month,4,3)

def get_E6_expiry(symbol):
    monthcode_yy = symbol[2:]
    next_month = DICT_MONTH_CODE[monthcode_yy[0]] + 1
    year = 2000 + int(monthcode_yy[1:])
    if next_month>12:
        next_month = 1
        year += 1
    return datetime.datetime(year,next_month,1) - 7*bday_us

def get_CL_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    month = month -1
    if month<1:
        month = 12
        year = year - 1
    return datetime.datetime(year,month,26) - 7*bday_us

def get_NG_expiry(symbol):
    monthcode_yy = symbol[2:]
    month = DICT_MONTH_CODE[monthcode_yy[0]]
    year = 2000 + int(monthcode_yy[1:])
    return datetime.datetime(year,month,1) - 4*bday_us

DICT_PRODUCT = {
    'E6':get_E6_expiry,
    'ES':get_ES_expiry,
    'CL':get_CL_expiry,
    'NG':get_NG_expiry,
}

    
def get_expiry(symbol):
    product = symbol[:2]
    f = DICT_PRODUCT[product]
    return f(symbol)


def dt_from_yyyymmdd(yyyymmdd,hour=0,minute=0,timezone=TIMEZONE):
    y = int(str(yyyymmdd)[0:4])
    m = int(str(yyyymmdd)[4:6])
    d = int(str(yyyymmdd)[6:8])  
    return datetime.datetime(y,m,d,hour,minute,tzinfo=pytz.timezone(timezone))

def yyyymmdd_from_dt(dt):
    y = int(dt.year)
    m = int(dt.month)
    d = int(dt.day)
    return y*100*100 + m*100 + d

def get_dte_pct(trade_yyyymmdd,expiry_yyyymmdd):
    dt_td = dt_from_yyyymmdd(trade_yyyymmdd)
    dt_xp = dt_from_yyyymmdd(expiry_yyyymmdd)
    return ((dt_xp - dt_td).days + 1)/365



### check dates vs last records in db for CL and ES

In [ ]:
# pg.PgPandas??
pga = pg.PgPandas(dburl='127.0.0.1',username='',password='',databasename='sec_db')

In [ ]:
syms = ['NG'+m+yy for m in 'FGHJKMNQUVXZ' for yy in ['15','16','17','18','19']]
for s in syms:
    ge = str(get_expiry(s))
    exp1 = int(ge[0:4])*100*100 + int(ge[5:7])*100 + int(ge[8:10])
    
    sql = f'''
    select max(ot.settle_date) from sec_schema.options_table ot
    where ot.symbol='{s}' 
    ;
    '''
    df_options = pga.get_sql(sql)
    exp2 = df_options.iloc[0]['max']
    print(s,exp1,exp2,exp1==exp2)
    #assert(exp1==exp2)

In [ ]:
get_rate(1)#,rate_datetime=datetime.datetime(2020,1,1))

In [ ]:
rdf = get_rate_df(1,datetime.datetime(2018,8,1),datetime.datetime(2018,8,31))
rdf

### Paremeters to calculate SPX based SKEW

In [ ]:
SKEW_TICKER = '^SPX' # yahoo symbol


In [ ]:
df_skew = pd.read_csv('https://www.cboe.com/publish/scheduledtask/mktdata/datahouse/skewdailyprices.csv',header=1)

In [ ]:
df_skew['date_split'] = df_skew.Date.apply(lambda s:s.split('/'))
df_skew['yyyymmdd'] = df_skew.date_split.apply(lambda s:int(s[2])*100*100 + int(s[0])*100 + int(s[1]))
df_skew

In [ ]:
def get_closest_expiry_to_dt(dt=None,options_ticker=SKEW_TICKER):
    nn = dt
    if nn is None:
        nn = datetime.datetime.now()
    spx = yf.Ticker(options_ticker)
    dt_nth_weekday = get_nth_weekday(int(nn.year),int(nn.month),4,3)
    spx_expirys = [datetime.datetime(int(s[0:4]),int(s[5:7]),int(s[8:10])) for s in spx.options]
    diffs = [abs((se - dt_nth_weekday).days) for se in spx_expirys]
    index_of_closes_expiry = diffs.index(min(diffs))
    ret_exp = spx_expirys[index_of_closes_expiry]
    return ret_exp
def get_current_spx_3rd_friday_exp():
    nn = datetime.datetime.now()
    exp1 = get_closest_expiry_to_dt(dt=nn)
    dt_2nd_month = nn + relativedelta(months=+1)
    exp2 = get_closest_expiry_to_dt(dt=dt_2nd_month)
    dt_3rd_month = dt_2nd_month + relativedelta(months=+1)
    exp3 = get_closest_expiry_to_dt(dt=dt_3rd_month)    
    if (exp1 - nn).days < 10:
        return (exp2,exp3)
    return (exp1,exp2)
get_current_spx_3rd_friday_exp()

### Parmeters related to fetching current values of SPX

In [ ]:
THRESHOLD_BID = .1 # minimum bid value when screening options
DELTA_K = 5 # strike difference between most of the options that get used to calculate SKEW
TRADE_DATE = 20191224
EXP_NEAR_YYYYMMDD = 20200116
EXP_FAR_YYYYMMDD = 20200220
INTEREST_RATE = (get_rate(1) + get_rate(2))/2 # calculate at run time ( this will be the average of the 1 month and 2 month libor rates from FRED)

OUTPUT_COLS = ['contractSymbol','strike','mid','dte_pct','ert',
                'forward_price','deltak','p1','p2','p3','e1','e2','e3']

DICT_THRESHOLD_BIDS = {'ES':1,'CL':.02,'CB':.02,'NG':.002}
DICT_DELTAK = {'ES':5,'CL':.5,'CB':.5,'NG':.1}


In [ ]:
ed = str(EXP_FAR_YYYYMMDD)
dd = f'{ed[0:4]}-{ed[4:6]}-{ed[6:8]}'
c = yf.Ticker('^SPX')
df_t = c.option_chain(dd)
              

In [ ]:
df_tp = df_t.puts[(df_t.puts.contractSymbol.str.slice(0,4)=='SPX2')]
df_tp[df_tp.strike>=3220.0].iloc[:3]

In [ ]:
df_tc = df_t.calls[(df_t.calls.bid>.1) & (df_t.calls.contractSymbol.str.slice(0,4)=='SPX2')]
df_tc[~df_tc.inTheMoney].iloc[:3]

In [ ]:

def get_valid_series_from_yfinance(trade_date_yyyymmdd,expiry_yyyymmdd,ticker=SKEW_TICKER,threshold_bid=THRESHOLD_BID,interest_rate=None,deltak=DELTA_K):
#     # ****** Step 01: create interest rate to use if necessary
    assert(interest_rate is not None)
    ir = interest_rate
#     if ir is None:
#         ir  = (get_rate(1) + get_rate(2))/2
    # ******  Step 02:  get yfinance contract, and options series
    contract = yf.Ticker(ticker)
    # reformat expiry_yyyymmdd into something like 2019-12-20
    ed = str(expiry_yyyymmdd)
    date_string = f'{ed[0:4]}-{ed[4:6]}-{ed[6:8]}'
    # get the options chain from yahoo finance
    df_opt = contract.option_chain(date_string)

    # ****** Step 03: figure out forward price
    df_spx_p = df_opt.puts[(df_opt.puts.bid>threshold_bid) & (df_opt.puts.contractSymbol.str.slice(0,4)=='SPX2')].copy()
    df_spx_p['cp'] = 'p'
    
    # ******  Step 03:  extract out of the money puts
    df_spx_p = df_spx_p[~df_spx_p.inTheMoney]
    df_spx_p = df_spx_p.sort_values('strike')
    df_spx_p.index = range(len(df_spx_p))

    # ******  Step 04: extract out the ATM and the out of the money calls
    df_spx_c = df_opt.calls[(df_opt.calls.bid>threshold_bid) & (df_opt.calls.contractSymbol.str.slice(0,4)=='SPX2')].copy()
    return df_spx_c
    if len(df_spx_c) < 1:
        # if no bids, use strikes up to 10% out of the money
        last_put_strike = df_spx_p.iloc[-1].strike
        last_call_strike = round(last_put_strike*1.1,0)
        df_spx_c = df_opt.calls[(df_opt.calls.strike<=last_call_strike) & (df_opt.calls.contractSymbol.str.slice(0,4)=='SPX2')].copy()
        return df_spx_c
    df_spx_c['cp'] = 'c'
    df_spx_c = df_spx_c[~df_spx_c.inTheMoney]
    df_spx_c = df_spx_c.sort_values('strike')
    df_spx_c.index = range(len(df_spx_c))
    first_call_strike = df_spx_c.iloc[0].strike
    # save the lowest_itm_put for later calc of forward price
    call_strikes = df_spx_c.strike.values
    lowest_itm_put = df_spx_p[(df_spx_p.strike>=first_call_strike) & df_spx_p.strike.isin(call_strikes)].iloc[0]
    
    

    # ******  Step 05: merge puts and alls into one dataframe
    df_ret = df_spx_p.copy()
    df_ret = df_ret.append(df_spx_c)
    df_ret = df_ret.sort_values(['cp','strike'])
    df_ret.index = range(len(df_ret))
    df_ret['expiry_yyyymmdd'] = expiry_yyyymmdd
    df_ret['trade_date_yyyymmdd'] = trade_date_yyyymmdd
        
    # ******  Step 06: Add mid price, expiry, dte_pct, ert, forward price     
    df_ret['mid'] = (df_ret.bid + df_ret.ask)/2
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    # find forward price by using the lowest strike call in df_ret 
    #     (what the SKEW whitepaper calls the At The Money option)
    # get ATM call midpoint
    atm_strike = lowest_itm_put.strike 
    
    atm_call = df_ret[(df_ret.strike==atm_strike) & (df_ret.cp=='c')].iloc[0]
    atm_call_price = (atm_call.bid + atm_call.ask) / 2 
    atm_put = lowest_itm_put
    atm_put_price = (atm_put.bid + atm_put.ask) / 2 
    forward_price = ert*(atm_call_price - atm_put_price) + atm_strike
    df_ret['forward_price'] = forward_price
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    # create p1 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3
    return df_ret
    

In [ ]:
exp1,exp2 = get_current_spx_3rd_friday_exp()
td = datetime.datetime.now() - 7*bday_us
exp1_yyyymmdd = yyyymmdd_from_dt(exp1)
exp2_yyyymmdd = yyyymmdd_from_dt(exp2)

ir_now = (get_rate(1,td) + get_rate(2,td))/2 # calculate at run time ( this will be the average of the 1 month and 2 month libor rates from FRED)

df_series_near = get_valid_series_from_yfinance(td,exp1_yyyymmdd,interest_rate=ir_now)
df_series_far = get_valid_series_from_yfinance(td,exp2_yyyymmdd,interest_rate=ir_now)


In [ ]:
# df_series_far[df_series_far.cp=='c'].iloc[0:1]

In [ ]:
# df_series_near[df_series_near.cp=='p'].iloc[-1:]

### sum up p1's p2's and p3's and computer e1, e2, and e3', and create SKEW

In [ ]:
def create_skew(df_series):
    e1 = df_series.iloc[0].e1
    e2 = df_series.iloc[0].e2
    e3 = df_series.iloc[0].e3
    ert = df_series.iloc[0].ert
    p1 = ert * -1 * df_series.p1.sum() + e1
    p2 = ert * df_series.p2.sum() + e2
    p3 = ert * df_series.p3.sum() + e3
    S_ =  (p3 - 3*p1*p2 + 2*p1**3) / (p2 - p1**2)**(3/2)
    SKEW = 100 - 10*S_
    return SKEW

In [ ]:
# Tnear = df_series_near.iloc[0].dte_pct
# Tfar = df_series_far.iloc[0].dte_pct
# T30 = 30/365
# w = (Tfar-T30)/(Tfar-Tnear)
# skew_near = create_skew(df_series_near)
# skew_far = create_skew(df_series_far)
# final_skew = w*skew_near + (1-w) * skew_far
# print(final_skew,skew_near,skew_far,w)

### now do multiple days from Oct 2015

In [ ]:
def get_valid_series_from_csv_file_df(df,threshold_bid=THRESHOLD_BID,interest_rate=INTEREST_RATE,deltak=DELTA_K):
    '''
    df: a DataFrame containing options settlements for a single day, and a single expiry
    '''
    # ****** Step 01: create interest rate to use if necessary
    ir = interest_rate
#     if ir is None:
#         ir  = (get_rate(1) + get_rate(2))/2
    
    df_ret = df.copy()
    trade_date_yyyymmdd = df_ret.trade_date_yyyymmdd.iloc[0]
    expiry_yyyymmdd = df_ret.expiry_yyyymmdd.iloc[0]
    assert(len(df_ret.expiry_yyyymmdd.unique())==1)
    # ******  Step 02: Add mid price, expiry, dte_pct, ert, forward price     
    df_ret['mid'] = (df_ret.bid + df_ret.ask)/2
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    lowest_atm_call = df_ret[(df_ret.underlying_last<df_ret.strike) & (df_ret.cp=='c')].sort_values('strike').iloc[0]
    atm_call_price = lowest_atm_call.mid
    atm_strike = lowest_atm_call.strike
    atm_put_price = df_ret[(df_ret.strike==atm_strike) & (df_ret.cp=='p')].iloc[0].mid
    forward_price = ert*(atm_call_price - atm_put_price) + atm_strike
    
    # ******  Step 03: Limit options to out of the money puts, the ATM call and out of the money calls
    df_ret_c = df_ret[(df_ret.cp=='c') & (df_ret.strike>=atm_strike)]
    df_ret_p = df_ret[(df_ret.cp=='p') & (df_ret.strike<atm_strike)]
    df_ret = df_ret_c.append(df_ret_p).sort_values(['cp','strike'])
    # ******  Step 04: make sure options have minimum bid
    df_ret = df_ret[df_ret.bid>threshold_bid]
    
    
    df_ret['forward_price'] = forward_price
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    # create p1 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3

    return df_ret
    

In [ ]:
def add_days_to_yyyymmdd(yyyymmdd,days_to_add):
    s = str(yyyymmdd)
    y = int(s[0:4])
    m = int(s[4:6])
    d = int(s[6:8])
    dt = datetime.datetime(y,m,d) + datetime.timedelta(days_to_add)
    ret = int(dt.year)*100*100 + int(dt.month)*100 + int(dt.day)
    return ret

In [ ]:
home =  pathlib.Path.home()
df_spx_oct_2015= pd.read_csv(f'{home}/downloads/spx_20151001_to_20151030.csv')
df_spx_oct_2015 = df_spx_oct_2015[df_spx_oct_2015.underlying=='SPX']
df_spx_oct_2015['cp'] = df_spx_oct_2015['type'].apply(lambda v:v[0])
df_spx_oct_2015['trade_date_yyyymmdd'] = df_spx_oct_2015.quotedate.apply(lambda s:s.split('/'))
df_spx_oct_2015['trade_date_yyyymmdd'] = df_spx_oct_2015.trade_date_yyyymmdd.apply(lambda s:(2000 + int(s[2]))*100*100 + int(s[0])*100 + int(s[1]))
df_spx_oct_2015['expiry_yyyymmdd'] = df_spx_oct_2015.expiration.apply(lambda s:s.split('/'))
df_spx_oct_2015['expiry_yyyymmdd'] = df_spx_oct_2015.expiry_yyyymmdd.apply(lambda s:(2000 + int(s[2]))*100*100 + int(s[0])*100 + int(s[1]))

df_spx_oct_2015.tail()


In [ ]:
df_spx_oct_2015[df_spx_oct_2015.trade_date_yyyymmdd==20151001]

### Get unique quotedate's

In [ ]:
trade_dates_yyyymmdd = df_spx_oct_2015.trade_date_yyyymmdd.unique()
dict_df_exp_pair = {}
for d in trade_dates_yyyymmdd:
    first_expiry_cutoff = add_days_to_yyyymmdd(d,10)
    df_spx_1 = df_spx_oct_2015[(df_spx_oct_2015.trade_date_yyyymmdd==d) & (df_spx_oct_2015.expiry_yyyymmdd>first_expiry_cutoff)]
    first_2_expiries = df_spx_1.expiry_yyyymmdd.unique()[:2]
    df_spx_exp1 = df_spx_1[df_spx_1.expiry_yyyymmdd==first_2_expiries[0]]
    df_spx_exp1_with_all_fields = get_valid_series_from_csv_file_df(df_spx_exp1)
    df_spx_exp2 = df_spx_1[df_spx_1.expiry_yyyymmdd==first_2_expiries[1]]
    df_spx_exp2_with_all_fields = get_valid_series_from_csv_file_df(df_spx_exp2)

    Tnear = df_spx_exp1_with_all_fields.iloc[0].dte_pct
    Tfar = df_spx_exp2_with_all_fields.iloc[0].dte_pct
    T30 = 30/365
    w = (Tfar-T30)/(Tfar-Tnear)
    skew_near = create_skew(df_spx_exp1_with_all_fields)
    skew_far = create_skew(df_spx_exp2_with_all_fields)
    final_skew = w*skew_near + (1-w) * skew_far
    actual_skew = df_skew[df_skew.yyyymmdd==d].iloc[0].SKEW
    print(d,actual_skew,final_skew,skew_near,skew_far,w,first_2_expiries[0],first_2_expiries[1])    

    dict_df_exp_pair[d] = {
        'df_spx_1':df_spx_exp1_with_all_fields,
                           'df_spx_2':df_spx_exp2_with_all_fields,
                           'skew_near':skew_near,
                           'skew_far':skew_far,
                           'final_skew':final_skew
    }
    


### Now use ES contract history from sec_db database

In [ ]:
s = 'CLZ18'
d = 20180801
sql = f'''
select * from sec_schema.options_table ot
where ot.settle_date>={d} 
and ot.symbol='{s}'
;
'''
df_options = pga.get_sql(sql)


sql = f'''
select * from sec_schema.underlying_table ft 
where ft.settle_date='{d}' 
and ft.symbol='{s}'
;
'''
df_futures = pga.get_sql(sql)
df_options['forward_price'] = df_futures.iloc[0].close
df_options['trade_date_yyyymmdd'] = df_options.settle_date

print(df_options)

In [ ]:
df_options

### Create date math functions to get options/futures expiry dates

In [ ]:
tt = f'./opvoct15/opv10015.csv'
import io
def _df_from_text(text_path):
    lines = open(text_path,'r').read()
    options_header = 'contract,month_year,strike_right,date,open,high,low,close,volume,open_interest'
    lines2 = options_header + '\n'+ lines
    lines3 = lines2.split('\n')
    s2 = io.StringIO()
    for l in lines3:
        s2.write(l+'\n')
    s2.seek(0)
    df = pd.read_csv(s2)
    return df
df_tt = _df_from_text(tt)

In [ ]:
[s for s in sorted(df_tt.contract.unique()) if s[0]=='E']

In [ ]:

df_tt[df_tt.contract.str.contains('ES')].month_year.unique()

In [ ]:
#" ".join(sorted(list(set([s[:4] for s in df_tt[df_tt.contract.str.contains('ES')].strike_right.unique()]))))

In [ ]:
# import datetime as dt

# from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, \
#     USMartinLutherKingJr, USPresidentsDay, GoodFriday, USMemorialDay, \
#     USLaborDay, USThanksgivingDay


# class USTradingCalendar(AbstractHolidayCalendar):
#     rules = [
#         Holiday('NewYearsDay', month=1, day=1, observance=nearest_workday),
#         USMartinLutherKingJr,
#         USPresidentsDay,
#         GoodFriday,
#         USMemorialDay,
#         Holiday('USIndependenceDay', month=7, day=4, observance=nearest_workday),
#         USLaborDay,
#         USThanksgivingDay,
#         Holiday('Christmas', month=12, day=25, observance=nearest_workday)
#     ]


# def get_trading_close_holidays(year):
#     inst = USTradingCalendar()

#     return inst.holidays(dt.datetime(year-1, 12, 31), dt.datetime(year, 12, 31))


# print(get_trading_close_holidays(2014)) 

### get ESX15 and ESZ15 options, ESZ15 underlying to 

In [ ]:
s = 'ESV15'
d = 20151001
sql = f'''
select * from sec_schema.options_table ot
where ot.symbol='{s}' and settle_date={d}
;
'''
df_options = pga.get_sql(sql)

f = 'ESZ15'
sql = f'''
select * from sec_schema.underlying_table ot
where ot.symbol='{f}'  and settle_date={d}
;
'''
df_futures = pga.get_sql(sql)
atm = df_futures.iloc[0].close



In [ ]:
df_options

In [ ]:
df_futures

In [ ]:
dfc = df_options[(df_options.pc.str.lower()=='c')][['settle_date','strike','close','pc']].sort_values('strike')
dfc = dfc[dfc.strike >= atm]
dfp = df_options[(df_options.pc.str.lower()=='p')][['settle_date','strike','close','pc']].sort_values('strike')
dfp = dfp[dfp.strike < atm]
dfb = dfc.append(dfp).sort_values(['pc','strike'])
dfb.pc = dfb.pc.str.lower()
dfb = dfb.rename(columns={'pc':'cp','settle_date':'trade_date_yyyymmdd'})
dfb = dfb[dfb.close >= 1 ]
dfb['expiry_yyyymmdd']  = get_expiry('ESV15')
dfb.strike.min(),dfb.strike.max(),len(dfb)

In [ ]:
dfp.tail()

### create SKEW from ESV15

In [ ]:
ir_20151001 = get_rate(1,dt_from_yyyymmdd(20151001))

In [ ]:
dict_fut_mon = {'F':'H','G':'H','H':'H','J':'M','K':'M','M':'M','N':'U','Q':'U','U':'U','V':'Z','X':'Z','Z':'Z'}
def get_valid_series_from_barchartacs(symbol,trade_date_yyyymmdd,
                                      interest_rate=None,deltak=DELTA_K):
    threshold_bid = DICT_THRESHOLD_BIDS[symbol[:2]]
#     print(symbol,trade_date_yyyymmdd)
    ir = interest_rate 
    if ir is None:
        ir = get_rate(1,dt_from_yyyymmdd(trade_date_yyyymmdd))
        
#     # ****** Step 01: get futures data from sql
    fm = dict_fut_mon[symbol[2]]
    futures_symbol = symbol[0:2] + fm + symbol[3:]
    sql = f'''
    select * from sec_schema.underlying_table ot
    where ot.symbol='{futures_symbol}'  and settle_date={trade_date_yyyymmdd}
    ;
    '''
    df_futures = pga.get_sql(sql)
    forward_price = df_futures.iloc[0].close

#     # ****** Step 02: get options data from sql
    sql = f'''
    select * from sec_schema.options_table ot
    where ot.symbol='{symbol}'  and settle_date={trade_date_yyyymmdd}
    ;
    '''
    df_options = pga.get_sql(sql)
    dfc = df_options[(df_options.pc.str.lower()=='c')][['settle_date','strike','close','pc']].sort_values('strike')
    dfc = dfc[dfc.strike >= forward_price]
    dfp = df_options[(df_options.pc.str.lower()=='p')][['settle_date','strike','close','pc']].sort_values('strike')
    dfp = dfp[dfp.strike < forward_price]
    atm_strike = dfc[dfc.strike == dfc.strike.min()].iloc[0].strike
    
    # ******  Step 03: merge puts and alls into one dataframe
    dfb = dfc.append(dfp).sort_values(['pc','strike'])    
    dfb.pc = dfb.pc.str.lower()
    dfb = dfb.rename(columns={'pc':'cp','settle_date':'trade_date_yyyymmdd','close':'mid'})
    dfb = dfb[dfb.mid >= threshold_bid ]
    exp_dt  = get_expiry(symbol)
    expiry_yyyymmdd = int(exp_dt.year)*100*100 + int(exp_dt.month)*100 + int(exp_dt.day)
    dfb['expiry_yyyymmdd'] = expiry_yyyymmdd
    df_ret = dfb.copy()
    df_ret['trade_date_yyyymmdd'] = trade_date_yyyymmdd
    df_ret['forward_price'] = forward_price
    
    df_ret = df_ret.sort_values(['cp','strike'])
    df_ret.index = range(len(df_ret))

    # ******  Step 04: Add m dte_pct, ert  
    dte_pct = get_dte_pct(trade_date_yyyymmdd,expiry_yyyymmdd)
    df_ret['dte_pct'] = dte_pct
    ert = np.exp(dte_pct * ir)
    df_ret['ert'] = ert
    
    df_ret['k_over_fp'] = df_ret.strike / forward_price
    df_ret['deltak'] = deltak
    
    # ****** Step 05: create p1,p2,p3, e1,e2,e3 unit values
    df_ret['p1']= df_ret.apply(lambda r: r.deltak / r.strike**2 * r.mid,axis=1)
    df_ret['p2'] = df_ret.apply(lambda r: 2 * r.p1 *(1-np.log(r.k_over_fp)),axis=1)
    df_ret['p3'] = df_ret.apply(lambda r: 3 * r.p1 * (2*np.log(r.k_over_fp) - np.log(r.k_over_fp)**2),axis=1)
    e1 = -(1+np.log(forward_price/atm_strike) - forward_price/atm_strike)
    e2 = 2 * np.log(atm_strike/forward_price) * (forward_price/atm_strike - 1) + 1/2 * np.log(atm_strike/forward_price)**2
    e3 = 3 * np.log(atm_strike/forward_price)**2 * (1/3 * np.log(atm_strike/forward_price) - 1 + forward_price/atm_strike)
    df_ret['e1'] = e1
    df_ret['e2'] = e2
    df_ret['e3'] = e3
    return df_ret
    

In [ ]:
get_valid_series_from_barchartacs('NGM19',20190401,interest_rate=.02,deltak=.1)


### Loop through a set of days and calculate skew

In [ ]:
trade_date_yyyymmdd_beg = 20180701
trade_date_yyyymmdd_end = 20180810
df_rates = get_rate_df(1,trade_date_yyyymmdd_beg,trade_date_yyyymmdd_end)
symbol='ESU18'
futures_symbol = 'ES' + dict_fut_mon[symbol[2]] + symbol[3:]
sql = f'''
select * from sec_schema.underlying_table ot
where ot.symbol='{futures_symbol}'  and settle_date>={trade_date_yyyymmdd_beg} and settle_date <= {trade_date_yyyymmdd_end}
;
'''
trade_dates_yyyymmdd = pga.get_sql(sql).settle_date.values
dates = []
actual_skews = []
final_skews  = []
dict_df_exp_pair = {}
for d in trade_dates_yyyymmdd:
    ir = df_rates[df_rates.date_yyyymmdd==d].iloc[0].fixed_rate
    dfes = get_valid_series_from_barchartacs(symbol,d,interest_rate=ir,deltak=5)
    final_skew = create_skew(dfes)
    actual_skew = df_skew[df_skew.yyyymmdd==d].iloc[0].SKEW
    print(d,actual_skew,final_skew) 
    dates.append(d)
    actual_skews.append(actual_skew)
    final_skews.append(final_skew)
    dict_df_exp_pair[d] = {
        'df_spx_1':dfes.copy(),
                           'final_skew':final_skew
    }
df_pga_skew = pd.DataFrame({'date':dates,'actual_skew':actual_skews,'final_skew':final_skews})    


In [ ]:
df_pga_skew

In [ ]:
iplot(cvt.plotly_plot(df_pga_skew,'date',plot_title='skew'))

In [ ]:
def create_skew_df(symbol,beg_yyyymmdd, end_yyyymmdd,deltak=5):
    trade_date_yyyymmdd_beg = beg_yyyymmdd
    trade_date_yyyymmdd_end = end_yyyymmdd
    df_rates = get_rate_df(1,trade_date_yyyymmdd_beg,trade_date_yyyymmdd_end)
    futures_symbol = symbol[:(len(symbol)-3)] + dict_fut_mon[symbol[2]] + symbol[3:]
    sql = f'''
    select * from sec_schema.underlying_table ot
    where ot.symbol='{futures_symbol}'  and settle_date>={trade_date_yyyymmdd_beg} and settle_date <= {trade_date_yyyymmdd_end}
    ;
    '''
    trade_dates_yyyymmdd = pga.get_sql(sql).settle_date.values
    dates = []
    final_skews  = []
    dict_df_exp_pair = {}
    for d in trade_dates_yyyymmdd:
        ir = df_rates[df_rates.date_yyyymmdd==d].iloc[0].fixed_rate
        dfes = get_valid_series_from_barchartacs(symbol,d,interest_rate=ir,deltak=deltak)
        final_skew = create_skew(dfes)
        dates.append(d)
        final_skews.append(final_skew)
        dict_df_exp_pair[d] = {
            'df_spx_1':dfes.copy(),
                               'final_skew':final_skew
        }
    df_pga_skew = pd.DataFrame({'date':dates,'final_skew':final_skews})    
    return df_pga_skew

In [ ]:
get_valid_series_from_barchartacs('CLF20',20191001,interest_rate=get_rate(1,datetime.datetime(2019,10,1)),deltak=.1)

In [ ]:
ss = 'NGM19'
dfs = create_skew_df(ss,20190321,20190422,deltak=.1)
iplot(cvt.plotly_plot(dfs,'date',plot_title=f'{ss} skew'))

### create dash app

#### define cell styles

In [ ]:
CONTRACTS_TO_DISPLAY_DICT = {'names':['E-Mini SP','Nymex Crude','Ice Brent','NYMEX Natural Gas'], 
                             'symbols':['ES','CL','CB','NG']
}                             

# Create css styles for some parts of the display

STYLE_TITLE={
    'line-height': '20px',
    'textAlign': 'center',
    'background-color':'#47bacc',
    'color':'#FFFFF9',
    'vertical-align':'middle',
} 

STYLE_UPGRID = STYLE_TITLE.copy()
STYLE_UPGRID['background-color'] = '#EAEDED'
STYLE_UPGRID['line-height'] = '10px'
STYLE_UPGRID['color'] = '#21618C'
STYLE_UPGRID['height'] = '50px'

ALL_SYMBOL_SQL = 'select distinct symbol from sec_schema.options_table;'
ALL_SYMBOLS = pga.get_sql(ALL_SYMBOL_SQL).symbol.values
ALL_PRODUCTS = sorted(list(set([s[:2] for s in ALL_SYMBOLS])))


In [ ]:
def get_all_years_per_product(product):
    global ALL_SYMBOLS
    return sorted(list(set([s[3:] for s in ALL_SYMBOLS if s[:2] == product])))
def get_all_monthcodes_per_product(product,year):
    global ALL_SYMBOLS
    yy = str(year)[-2:]
    return sorted(list(set([s[2] for s in ALL_SYMBOLS if (s[:2] == product) & (s[-2:]==yy)])))
def get_dates_per_symbol(symbol):
    sql_dates = f'''
    select min(settle_date) min_date, max(settle_date) max_date 
    from sec_schema.options_table 
    where symbol='{symbol}'; 
    '''
    df_dates_per_symbol = pga.get_sql(sql_dates)
    min_date_yyyymmdd = int(df_dates_per_symbol.iloc[0].min_date)
    max_date_yyyymmdd = int(df_dates_per_symbol.iloc[0].max_date)
    # subtract 11 days from max_date_yyyymmdd
    max_date_yyyymmdd = yyyymmdd_from_dt(dt_from_yyyymmdd(max_date_yyyymmdd) - datetime.timedelta(11))
    return [min_date_yyyymmdd,max_date_yyyymmdd]

In [ ]:
get_dates_per_symbol('CLM19')

#### define cells

In [ ]:
list(range(2011,2021))

In [ ]:
# DEFAULT_PROD = 'CL'
# DEFAULT_YEAR = 2020
# # Step 1: create a title at the top
# title_div = html.Div([html.H3('Commodity Options SKEW INDEX Analysis')],style=STYLE_TITLE)

# # Step 2: create a grid that includes an explanation, and 2 dropdowns
# #    Step 2.1: create the information h3
# m = html.H3("From the dropdown buttons to the right, select a Commodity and/or a Year",style={'height':'1px'})
# info_div = dgrid.GridItem(m,html_id='explain')
# select_commod_div  = dgrid.DropDownDiv('commod_dropdown', 
#                         CONTRACTS_TO_DISPLAY_DICT['names'], 
#                          CONTRACTS_TO_DISPLAY_DICT['symbols'],style=STYLE_UPGRID,
#                         transformer_method=lambda data: data#_transform_commod_selection
#                          )
# init_whole_years = [str(i) for i in list(range(2011,2021))]
# init_yy = [str(i)[-2:] for i in init_whole_years]
# select_year_div =  dgrid.DropDownDiv('year_dropdown',                         
#                         init_whole_years,init_yy,
#                         style=STYLE_UPGRID,default_initial_index=len(init_whole_years)-1)



# init_months = [s for s in 'FGHJKMNQUVXZ']
# select_month_div =  dgrid.DropDownDiv('month_dropdown',                         
#                         init_months,init_months,
#                         style=STYLE_UPGRID,default_initial_index=len(init_months)-1)

# dropdown_grid = dgrid.create_grid([info_div,select_commod_div,select_year_div,select_month_div],num_columns=4,column_width_percents=[55,14.95,14.95,14.95])


#### define main grid


In [ ]:
# def get_symbol_to_show():
#     prod = select_commod_div.current_value
#     month = select_month_div.current_value
#     yy = select_year_div.current_value
#     sym = prod+month+yy
#     print(f'get_symbol_to_show: {sym}')
#     # get all dates for this sym
#     beg_end_yyyymmdds = get_dates_per_symbol(sym)
#     # get last date
#     end_yyyymmdd = beg_end_yyyymmdds[1]
#     # make first date 60 days back from last date
#     dt_beg = dt_from_yyyymmdd(end_yyyymmdd) - datetime.timedelta(60)
#     beg_yyyymmdd = yyyymmdd_from_dt(dt_beg)
#     deltak = DICT_DELTAK[prod]
#     dfs = create_skew_df(sym,beg_yyyymmdd,end_yyyymmdd,deltak=deltak)
#     fig = cvt.plotly_plot(dfs,'date',plot_title=f'{sym} skew')
#     gr = dgrid.GridGraph(fig.layout.title, fig.layout.title ,None,figure=fig,
#             df_x_column='date')
#     # combine the table and the graph into the main grid
#     main_grid =  dgrid.create_grid([gr],num_columns=1)
#     return main_grid

# content_div = dgrid.ReactiveDiv('page_content',select_commod_div.output_tuple,
# #                     input_transformer=lambda commod,data:main_grid,
#                     input_transformer=lambda commod,data:get_symbol_to_show(),
#                     dom_storage_dict=None)


#### define main app

In [ ]:
# app = dash.Dash(url_base_pathname='/skew/')
# main_div = html.Div(children=[title_div,dropdown_grid,content_div.html])

# app.layout = html.Div(children=[main_div])

# callback_components = [select_commod_div,select_year_div,select_month_div,content_div]
# [c.callback(app) for c in callback_components]


# # Step 5: run the server    
# host = '127.0.0.1'
# port = 8600
# app.run_server(host=host,port=port)


### Create Dash App to server up SKEW

In [ ]:
sys.path.append(os.path.abspath('../../risktables'))
sys.path.append(os.path.abspath('../../risktables/risktables'))


In [ ]:
from risktables import dgrid_components as dgc

In [ ]:
import importlib
importlib.reload(dgc)
# dgc.XyGraphComponent??

In [ ]:
get_all_monthcodes_per_product('ES',2014)

In [ ]:
def show_data(sym):
    global DICT_DELTAK
    # get all dates for this sym
    beg_end_yyyymmdds = get_dates_per_symbol(sym)
    # get last date
    end_yyyymmdd = beg_end_yyyymmdds[1]
    # make first date 60 days back from last date
    dt_beg = dt_from_yyyymmdd(end_yyyymmdd) - datetime.timedelta(60)
    beg_yyyymmdd = yyyymmdd_from_dt(dt_beg)
    prod = sym[:2]
    deltak = DICT_DELTAK[prod]
    dfs = create_skew_df(sym,beg_yyyymmdd,end_yyyymmdd,deltak=deltak)
    return dfs


In [ ]:
logger = dgc.init_root_logger('logfile.log','WARN') 

top_div = html.Div([
                    dgc.dcc.Markdown('''
                    # Commodity Option SKEW Analysis
                    Select a Commodity, Year and Monthcode below. The resulting data is derived from the CBOE SKEW formula outlined in the whitepaper: 
                    
                    (https://www.cboe.com/micro/skew/documents/skewwhitepaperjan2011.pdf)
                    '''
                    ,style={'color':'white'})
            ],
            style=STYLE_TITLE,id='top_div')


dropdown_instructions = dgc.DivComponent('dd_instructions',initial_children=['Select from the Product, Year and Month Dropdowns'])

chained_dd_prods = dgc.ChainedDropDownDiv('chained_dd_prods',
                initial_dropdown_labels=['Emini','WTI Crude','Brent Crude'],
                initial_dropdown_values=['ES','CL','CB'])

def _chained_years(inputs):
    prod = inputs[1]
    if prod is None or len(prod)<1:
        return []
    yys = get_all_years_per_product(prod)
    choices = [{'label':str(2000 + int(yy)),'value':yy} for yy in yys]
    return  choices

    
chained_dd_years = dgc.ChainedDropDownDiv('chained_dd_years',
                dropdown_input_components=[chained_dd_prods],
                choices_transformer_method=_chained_years,
                placeholder="Select a year")

def _chained_months(inputs):
    if inputs is None or len(inputs)<3:
        return []
    prod = inputs[1]
    if prod is None or len(prod)<1:
        return []
    yy = inputs[2]
    if yy is None or len(yy)<1:
        return []
    year = 2000 + int(yy)
    mcs = get_all_monthcodes_per_product(prod,year)
    choices = [{'label':mc ,'value':mc} for mc in mcs]
    return  choices

    
chained_dd_months = dgc.ChainedDropDownDiv('chained_dd_months',
                dropdown_input_components=[chained_dd_prods,chained_dd_years],
                choices_transformer_method=_chained_months,
                placeholder="Select a month code")

full_symbol_store_inputs = [
    (chained_dd_prods.dropdown_id,'value'),
    (chained_dd_years.dropdown_id,'value'),
    (chained_dd_months.dropdown_id,'value'),    
]

def _create_full_symbol(inputs):
    print(f'_create_full_symbol inputs {inputs}')
    if inputs is None or len(inputs)<3 or inputs[0] is None or inputs[1] is None or inputs[2] is None:
        return {}
    prod = inputs[0]
    yy = str(inputs[1])[-2:]
    month = inputs[2]
    full_symbol = prod+month+yy
    full_symbol = full_symbol.upper()
    print(f'full_symbol {full_symbol}')
    dict_df = show_data(full_symbol).to_dict()
    return {'full_symbol':full_symbol,'df':dict_df}
    
full_symbol_store = dgc.StoreComponent('symbol_store',full_symbol_store_inputs,
                            create_data_dictionary_from_df_transformer=_create_full_symbol)

def _symbol_from_store(inputs):
    print(f'_symbol_from_store inputs: {inputs}')
    if inputs is None or len(inputs)<1:
        return ['']
    sym_dict = inputs[0]
    if sym_dict is None or len(sym_dict)<1:
        return ['']
    return [sym_dict['full_symbol']]

full_symbol_div = dgc.DivComponent('full_symbol_div',input_component=full_symbol_store,
                                  callback_input_transformer=_symbol_from_store)

def transform_input_to_df(dict_df,key_of_df,columns_to_show=None):
    df = None
    dict_this_risk = None
    if len(dict_df)>0:
        dict_this_risk = dict_df[key_of_df]
        df = dgc.make_df(dict_this_risk)
        if columns_to_show is not None:
            df = df[columns_to_show]
    return df


dash_graph = dgc.XyGraphComponent('dash_graph',full_symbol_store,'date',
                title="Skew Graph",plot_bars=False,
                transform_input=lambda dict_df: transform_input_to_df(dict_df,'df'))

dash_table = dgc.DashTableComponent('dash_table',None,input_component=full_symbol_store,
                title="Skew Data",
                transform_input=lambda dict_df: transform_input_to_df(dict_df,'df'),
                columns_to_round=[],digits_to_round=3)




In [ ]:
app_to_use = dash.Dash(url_base_pathname='/skew/')
# app.layout = html.Div(children=[chained_dd.html])

app_component_list = [top_div,dropdown_instructions,chained_dd_prods,
                      chained_dd_years,chained_dd_months,full_symbol_store,full_symbol_div,dash_graph,dash_table]

gtcl = ['1fr','4fr 1fr 1fr 1fr','0fr 1fr','1fr','1fr']
app = dgc.make_app(app_component_list,
                app=app_to_use,
                grid_template_columns_list=gtcl)    


# Step 5: run the server    
host = '127.0.0.1'
port = 8600
app.run_server(host=host,port=port)


## End